In [1]:
from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.applications.vgg16 import VGG16

from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

In [2]:
(x_train, y_train), (x_test, y_test) = load_data()

In [3]:
x_mean = np.mean(x_train, axis=(0, 1, 2))
x_std = np.std(x_train, axis=(0, 1, 2))

In [4]:
x_train = (x_train - x_mean) / x_std
x_test = (x_test - x_mean) / x_std

In [5]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=1)

In [6]:
# transfer learning (전이 학습) : pre-trained (사전 학습) 되어 있는 모델을 가지고 와서 사용하고 싶다.
# VGG 16 : Visul Geometry Group 16개 layer
# weights="imagenet" : imagenet datasets 로 학습된 가중치를 사용하겠다 (pre-trained)
# include_top=False : fc layer를 사용하지 않겠다
vgg16 = VGG16(input_shape=(32, 32, 3), weights="imagenet", include_top=False)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [7]:
# 13 개의 covolution layer + 3개의 fc layer
# fully connected layer
vgg16.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# 마지막 4개의 layer를 제외하고, 나머지 layer들은 학습하지 않겠다 (= w, b 변경하지 않겠다)
for layer in vgg16.layers[:-4]:
    layer.trainable = False
    # 계속 바꿔가면서 어떨때 성능이 좋은지 확인 가능

In [12]:
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256))
# BatchNormalization : layer의 입력을 평균이 0, 분산이 1이 되도록 정규화 -> 데이터의 입력 분포가 일관되게 유지 = 학습 속도가 빨라짐
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dense(10, activation="softmax"))

In [13]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,849,610 (56.65 MB)

 Trainable params: 7,213,834 (27.52 MB)

 Non-trainable params: 7,635,776 (29.13 MB)

In [15]:
# adam running rate 지저
model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(0.001), metrics=["acc"])

In [16]:
# fine tuning : 이미 학습되어 있는 모델을, 우리의 데이터에 맞게 w, b 값을 미세하게 조정 (학습)
result = model.fit(x_train, y_train, epochs=10, callbacks=[EarlyStopping(patience=3)], validation_data=(x_val, y_val))

Epoch 1/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 353s 321ms/step - acc: 0.6202 - loss: 1.0919 - val_acc: 0.6368 - val_loss: 1.1266
Epoch 2/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 384s 323ms/step - acc: 0.7502 - loss: 0.7173 - val_acc: 0.7351 - val_loss: 0.7843
Epoch 3/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 377s 345ms/step - acc: 0.7841 - loss: 0.6260 - val_acc: 0.7293 - val_loss: 0.8022
Epoch 4/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 359s 328ms/step - acc: 0.8085 - loss: 0.5404 - val_acc: 0.7383 - val_loss: 0.8152
Epoch 5/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 361s 330ms/step - acc: 0.8426 - loss: 0.4528 - val_acc: 0.7623 - val_loss: 0.7288
Epoch 6/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 363s 332ms/step - acc: 0.8692 - loss: 0.3733 - val_acc: 0.7729 - val_loss: 0.7087
Epoch 7/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 361s 330ms/step - acc: 0.9013 - loss: 0.2839 - val_acc: 0.7751 - val_loss: 0.7362
Epoch 8/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 370s 338ms/step - acc: 0.9210 - loss: 0.2296 - val_acc: 0.7809 - val_loss: 0.7894
Epoch 9/

In [17]:
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 46s 147ms/step - acc: 0.7576 - loss: 0.9943


[0.9734914302825928, 0.7562999725341797]